In [2]:
import pandas as pd
# 读取肥胖数据
df_obesity = pd.read_csv('Obesity among adults by country/obesity-cleaned.csv')
# 删除男女分开的数据，保留both sexs.
df_obesity = df_obesity[~df_obesity['Sex'].isin(['Male', 'Female'])]
# 删除第一列（数据集自带的类似index的列）
df_obesity = df_obesity.drop(df_obesity.columns[0], axis=1)
# 删除sex列
df_obesity = df_obesity.drop(df_obesity.columns[3], axis=1)
# 删除Obesity的[]的部分
df_obesity['Obesity (%)'] = df_obesity['Obesity (%)'].str.split().str[0]
# 重新整理index
df_obesity=df_obesity.reset_index(drop=True)
df_obesity.head()

,Country,Year,Obesity (%)
0,Afghanistan,1975,0.5
1,Afghanistan,1976,0.5
2,Afghanistan,1977,0.6
3,Afghanistan,1978,0.6
4,Afghanistan,1979,0.6


In [3]:
# 读取国家代码表
df_CountryCode = pd.read_csv('Extra data needed/wikipedia-iso-country-codes.csv')
df_CountryCode = df_CountryCode.drop(['Alpha-2 code', 'Numeric code', 'ISO 3166-2'], axis=1)
df_CountryCode.head()

,English short name lower case,Alpha-3 code
0,Zimbabwe,ZWE
1,Zambia,ZMB
2,Yemen,YEM
3,Western Sahara,ESH
4,Wallis and Futuna,WLF


In [4]:
# 将肥胖数据中的国家改为国家代码
df_obesity_merged = pd.merge(df_obesity, df_CountryCode, left_on='Country', right_on='English short name lower case')
df_obesity_merged.drop(['Country', 'English short name lower case'], axis=1, inplace=True)
df_obesity_merged.rename(columns={'Alpha-3 code': 'Country_Code'}, inplace=True)
df_obesity = df_obesity_merged
df_obesity.head()

,Year,Obesity (%),Country_Code
0,1975,0.5,AFG
1,1976,0.5,AFG
2,1977,0.6,AFG
3,1978,0.6,AFG
4,1979,0.6,AFG


In [18]:
#读取GDP数据
df_GDP = pd.read_csv('Country Economic Indicators/GDP.csv')
df_GDP = df_GDP.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year'})
# 删除不需要的列
df_GDP = df_GDP.drop(['INDICATOR', 'SUBJECT', 'FREQUENCY', 'Flag Codes'], axis=1)
# 使用 pivot_table 方法重塑数据框，使得MEASURE=MLN_USD和MEASURE=USD_CAP的两类数据对应的Value分开记作两列
df_GDP_reshaped = df_GDP.pivot_table(index=['Country_Code', 'Year'], columns='MEASURE', values='Value').reset_index().rename_axis(None, axis=1)
# 重命名列
'''MLN_USD: This likely stands for GDP in millions of U.S. dollars, which provides the total economic output of a country valued in USD.
USD_CAP: This typically refers to GDP per capita in U.S. dollars, which divides the country's total economic output by its population, reflecting the average economic output per person, adjusted to U.S. dollars.'''
df_GDP = df_GDP_reshaped.rename(columns={'MLN_USD': 'MLN_USD', 'USD_CAP': 'USD_CAP'})
df_GDP.head()


,Country_Code,Year,MLN_USD,USD_CAP
0,ALB,1997,8555.927,2726.202207
1,ALB,1998,9451.507,3030.636470
2,ALB,1999,10792.634,3482.728286
3,ALB,2000,11926.200,3874.176721
4,ALB,2001,13157.360,4299.547816


In [6]:
# 读取AVWAGE数据集
df_AVWAGE = pd.read_csv('Country Economic Indicators/Average_wage.csv')
df_AVWAGE = df_AVWAGE[['LOCATION', 'TIME', 'Value']]
df_AVWAGE = df_AVWAGE.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'AVWAGE'})
df_AVWAGE.head()

,Country_Code,Year,AVWAGE
0,AUS,1990,39875.345702
1,AUS,1991,39685.259630
2,AUS,1992,40481.994276
3,AUS,1993,40854.703847
4,AUS,1994,41131.575639


In [7]:
#读取FERTILITY数据集
df_FERTILITY = pd.read_csv('Country Economic Indicators/Fertility_rates.csv')
df_FERTILITY = df_FERTILITY[['LOCATION', 'TIME', 'Value']]
df_FERTILITY = df_FERTILITY.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'FERTILITY'})
df_FERTILITY.head()

,Country_Code,Year,FERTILITY
0,AUS,1960,3.45
1,AUS,1961,3.55
2,AUS,1962,3.43
3,AUS,1963,3.34
4,AUS,1964,3.15


In [11]:
# 读取Employment_rate数据集
df_EMP = pd.read_csv('Country Economic Indicators/Employment_rate.csv')
df_EMP = df_EMP.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'EMP'})
# 删除 'SUBJECT' 为 'MEN' 或 'WOMEN' 的行，以及 'MEASURE' 为 'THND_PER' 的行
df_EMP = df_EMP[~((df_EMP['SUBJECT'] == 'MEN') | (df_EMP['SUBJECT'] == 'WOMEN') | (df_EMP['MEASURE'] == 'THND_PER'))]
# 删掉这些列：INDICATOR SUBJECT MEASURE FREQUENCY Flag Codes
df_EMP = df_EMP.drop(columns=['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'])
df_EMP.head()

,Country_Code,Year,EMP
215,AUS,1979,64.38335
216,AUS,1980,65.17445
217,AUS,1981,65.36742
218,AUS,1982,64.16441
219,AUS,1983,61.94022


In [15]:
# 读取Unemployment_rate数据集
df_HUR = pd.read_csv('Country Economic Indicators/Unemployment_rate.csv')
df_HUR = df_HUR.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'HUR'})
# 删除 'SUBJECT' 为 'MEN' 或 'WOMEN' 的行
df_HUR = df_HUR[~((df_HUR['SUBJECT'] == 'MEN') | (df_HUR['SUBJECT'] == 'WOMEN') )]
# 删掉这些列：INDICATOR SUBJECT MEASURE FREQUENCY Flag Codes
df_HUR = df_HUR.drop(columns=['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'])
''' It is calculated as the labour force (people available to work and actively seeking employment) divided by the total working-age population, which typically includes people aged 15 to 64. '''
df_HUR.head()

,Country_Code,Year,HUR
0,AUS,1967,1.875
1,AUS,1968,1.850
2,AUS,1969,1.800
3,AUS,1970,1.625
4,AUS,1971,1.925


In [12]:
# 读取Hours_worked数据集（每年工作小时数）
df_HRWKD = pd.read_csv('Country Economic Indicators/Hours_worked.csv')
df_HRWKD = df_HRWKD[['LOCATION', 'TIME', 'Value']]
df_HRWKD = df_HRWKD.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'HRWKD'})
df_HRWKD.head()

,Country_Code,Year,HRWKD
0,AUS,1985,1876.0
1,AUS,1986,1877.0
2,AUS,1987,1889.0
3,AUS,1988,1885.0
4,AUS,1989,1875.0


In [16]:
# 读取General_Government_debt数据集
# It stands for "percentage of GDP
df_GGDEBT = pd.read_csv('Country Economic Indicators/General_Government_debt.csv')
df_GGDEBT = df_GGDEBT[['LOCATION', 'TIME', 'Value']]
df_GGDEBT = df_GGDEBT.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'GGDEBT'})
df_GGDEBT.head()

,Country_Code,Year,GGDEBT
0,AUS,1995,57.81464
1,AUS,1996,54.86277
2,AUS,1997,53.39208
3,AUS,1998,49.75021
4,AUS,1999,42.02913


In [17]:
# 读取Young_population数据集
df_YNGPOP = pd.read_csv('Country Economic Indicators/Young_population.csv')
df_YNGPOP = df_YNGPOP[['LOCATION', 'TIME', 'Value']]
df_YNGPOP = df_YNGPOP.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'YNGPOP'})
df_YNGPOP.head()

,Country_Code,Year,YNGPOP
0,AUS,1950,26.604473
1,AUS,1951,27.145351
2,AUS,1952,27.601459
3,AUS,1953,28.122696
4,AUS,1954,28.590664


In [22]:
#读取排放数据
df_MtCO2 = pd.read_csv('Emissions by Country/GCB2022v27_MtCO2_flat.csv')
df_MtCO2 = df_MtCO2[['ISO 3166-1 alpha-3', 'Year', 'Total']]
df_MtCO2 = df_MtCO2.rename(columns={'ISO 3166-1 alpha-3': 'Country_Code','Total': 'MtCO2'})
df_MtCO2.head()

,Country_Code,Year,MtCO2
0,AFG,1750,0.0
1,AFG,1751,0.0
2,AFG,1752,0.0
3,AFG,1753,0.0
4,AFG,1754,0.0


In [30]:
#清理合并数据框
# 转换函数
def convert_columns_to_str(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
    return df
# 转换 'Country_Code' 和 'Year' 列为字符串类型
df_obesity = convert_columns_to_str(df_obesity, ['Country_Code', 'Year'])
df_GDP = convert_columns_to_str(df_GDP, ['Country_Code', 'Year'])
df_AVWAGE = convert_columns_to_str(df_AVWAGE, ['Country_Code', 'Year'])
df_FERTILITY = convert_columns_to_str(df_FERTILITY, ['Country_Code', 'Year'])
df_EMP = convert_columns_to_str(df_EMP, ['Country_Code', 'Year'])
df_HUR = convert_columns_to_str(df_HUR, ['Country_Code', 'Year'])
df_HRWKD = convert_columns_to_str(df_HRWKD, ['Country_Code', 'Year'])
df_GGDEBT = convert_columns_to_str(df_GGDEBT, ['Country_Code', 'Year'])
df_YNGPOP = convert_columns_to_str(df_YNGPOP, ['Country_Code', 'Year'])
df_MtCO2 = convert_columns_to_str(df_MtCO2, ['Country_Code', 'Year'])
# 清理数据的函数
def clean_data(df, value_columns):
    if not isinstance(value_columns, list):
        value_columns = [value_columns]
    for col in value_columns:
        df = df[(df[col] != 0) & (df[col].notnull())]
    return df

# 清理每个数据框
df_obesity_clean = clean_data(df_obesity, 'Obesity (%)')
df_GDP_clean = clean_data(df_GDP, ['MLN_USD', 'USD_CAP'])
df_AVWAGE_clean = clean_data(df_AVWAGE, 'AVWAGE')
df_FERTILITY_clean = clean_data(df_FERTILITY, 'FERTILITY')
df_EMP_clean = clean_data(df_EMP, 'EMP')
df_HUR_clean = clean_data(df_HUR, 'HUR')
df_HRWKD_clean = clean_data(df_HRWKD, 'HRWKD')
df_GGDEBT_clean = clean_data(df_GGDEBT, 'GGDEBT')
df_YNGPOP_clean = clean_data(df_YNGPOP, 'YNGPOP')
df_MtCO2_clean = clean_data(df_MtCO2, 'MtCO2')

# 合并数据框
dfs = [df_GDP_clean, df_AVWAGE_clean, df_FERTILITY_clean, df_EMP_clean, df_HUR_clean, df_HRWKD_clean, df_GGDEBT_clean, df_YNGPOP_clean, df_MtCO2_clean]
df_merged = df_obesity_clean
for df in dfs:
    df_merged = df_merged.merge(df, on=['Country_Code', 'Year'], how='inner')

# 展示最终合并后的数据框
df_merged.head()



,Year,Obesity (%),Country_Code,MLN_USD,USD_CAP,AVWAGE,FERTILITY,EMP,HUR,HRWKD,GGDEBT,YNGPOP,MtCO2
0,1995,17.6,AUS,403410.089,22405.572327,40973.528964,1.82,67.62838,8.472281,1870.0,57.81464,21.525123,305.002996
1,1996,18.1,AUS,424062.042,23268.405809,42187.475279,1.80,67.66373,8.506114,1862.0,54.86277,21.369744,311.886129
2,1997,18.6,AUS,450299.854,24442.265328,43450.036024,1.78,67.36479,8.362488,1861.0,53.39208,21.214694,320.282630
3,1998,19.1,AUS,477686.538,25671.582448,44066.519950,1.76,67.80048,7.677429,1850.0,49.75021,21.047982,334.075978
4,1999,19.7,AUS,510213.964,27121.296361,45103.501485,1.76,68.23203,6.873791,1856.0,42.02913,20.890904,343.488633
